In [1]:
import sys
from typing import Literal

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import tensorflow as tf
from plotly.colors import n_colors
from tensorflow.keras.layers import GRU, Dense, Embedding, SimpleRNN, StringLookup
from tensorflow.keras.models import Model

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from equation_discover import *

2023-12-23 22:31:45.028353: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-23 22:31:45.091783: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-23 22:31:45.396868: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-23 22:31:45.396971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-23 22:31:45.459082: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
from scipy.optimize import basinhopping, minimize
from tensorflow.keras.losses import MSE

In [3]:
tree = Node.from_sequence(
    [
        BASE_TOKENS.symbols.index(value)
        for value in ["sin", "+", "*", "const", "var_x", "const"]
    ]
)

X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())

# TF

In [62]:
import tensorflow_probability as tfp
from tensorflow.keras.losses import MSE
import functools

In [63]:
X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())
model = EvalModel(tree)

In [70]:
model.constants = np.array([0.5, 0])

In [71]:
MSE(y, model.eval(X))

<tf.Tensor: shape=(), dtype=float32, numpy=0.99416155>

In [72]:
model({"X": X, "constants": np.random.randn(2)})

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([ 0.30863234,  0.2862835 ,  0.26377782,  0.24112763,  0.21834525,
        0.19544327,  0.17243417,  0.14933059,  0.12614518,  0.10289065,
        0.07957973,  0.05622523,  0.03283989,  0.00943656, -0.01397195,
       -0.03737276, -0.06075313, -0.08410019, -0.10740119, -0.13064332,
       -0.15381388, -0.17690013, -0.19988945, -0.22276926, -0.24552698,
       -0.26815018, -0.29062644, -0.31294343, -0.33508897, -0.35705087,
       -0.3788171 , -0.4003758 , -0.42171508, -0.4428233 , -0.46368885,
       -0.48430032, -0.5046464 , -0.524716  , -0.544498  , -0.5639817 ,
       -0.58315635, -0.60201144, -0.6205367 , -0.6387219 , -0.656557  ,
       -0.6740325 , -0.69113857, -0.70786595, -0.7242054 , -0.74014807],
      dtype=float32)>

In [73]:
model.optimize_constants(X, y, "basinhopping", niter=10000).constants

array([2.00001025, 1.00017798])

In [ ]:
model.fit(X, y)

In [ ]:
model({"X": X, "constants": model.constants})

In [ ]:
def make_val_and_grad_fn(value_fn):
    @functools.wraps(value_fn)
    def val_and_grad(x):
        return tfp.math.value_and_gradient(value_fn, x)

    return val_and_grad

In [ ]:
value_and_grads = make_val_and_grad_fn(
    lambda x: MSE(y, model({"X": X, "constants": x}))
)

In [ ]:
value_and_grads = model.function_factory(X, y)

In [ ]:
value_and_grads(tf.convert_to_tensor(np.random.rand(2)))

In [ ]:
results = tfp.optimizer.lbfgs_minimize(
    value_and_gradients_function=value_and_grads,
    initial_position=np.random.randn(2).astype(np.float32),
)

In [ ]:
results.position

In [ ]:
value_and_grads = model.function_factory(X, y)

In [ ]:
loss, grads = value_and_grads(model.constants)

In [ ]:
tf.math.is_finite(loss)

In [ ]:
tf.norm(grads)

In [ ]:
tfp.optimizer.lbfgs_minimize(
    value_and_gradients_function=value_and_grads,
    initial_position=tf.constant(model.constants),
)

# Expression

In [ ]:
expression = Expression(tree)

In [ ]:
results = {}
T = 1e-1
step_size = 1
for n in range(50):
    res = basinhopping(
        lambda constants: MSE(y, expression.eval(X, constants)),
        expression.constants,
        T=T,
        stepsize=step_size,
        niter=1000,
        # niter_success=50
    )
    results[(T, step_size, n)] = res.x

results = pd.Series(results).rename_axis(["T", "step_size", "n"]).to_frame("res")
results.reset_index(inplace=True)
results["mse"] = results.apply(
    lambda x: MSE(y, expression.eval(X, x.res)).numpy(), axis=1
)

In [ ]:
(results.mse < 1e-3).sum()

In [ ]:
results = {}
for step_size in np.logspace(-1, 1, 10):
    for T in np.logspace(-3, 3, 10):
        for n in range(30):
            res = basinhopping(
                lambda constants: MSE(y, expression.eval(X, constants)),
                expression.constants,
                T=T,
                stepsize=step_size,
                niter=500,
            )
            results[(T, step_size, n)] = res.x

results = pd.Series(results).rename_axis(["T", "step_size", "n"]).to_frame("res")
results.reset_index(inplace=True)
results["mse"] = results.apply(
    lambda x: MSE(y, expression.eval(X, x.res)).numpy(), axis=1
)

In [ ]:
px.imshow(
    results.groupby(["T", "step_size"]).apply(lambda x: (x.mse < 1e-1).sum()).unstack(),
    aspect="auto",
).update_layout(xaxis_type="log", yaxis_type="log")

In [ ]:
fig = go.Figure()

colors = n_colors(
    "rgb(5, 200, 200)", "rgb(200, 10, 10)", results["T"].nunique(), colortype="rgb"
)
for (T, group), color in zip(results.groupby("T"), colors):
    fig.add_violin(x=group.distance, name=f"{T:.03f}", line_color=color)
fig.update_traces(orientation="h", side="positive", points=False)
fig.update_layout(height=600)